In [1]:
#import statements

import numpy as np
import html
import re
from nltk import word_tokenize
from nltk import bigrams
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize.casual import TweetTokenizer


#import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence
from keras.utils.np_utils import to_categorical
from keras.models import load_model

import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Embedding, Input, Lambda
from keras.layers.wrappers import Bidirectional
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import SGD, Adagrad
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import MaxPooling1D, GlobalMaxPooling1D, AveragePooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.recurrent import LSTM, GRU
from keras.preprocessing import sequence

import numpy as np
import tensorflow as tf
import re
import keras.callbacks
import sys
import os


from scipy.stats import spearmanr, pearsonr
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures, scale
from sklearn.ensemble import AdaBoostRegressor
from scipy import stats

Using TensorFlow backend.


In [9]:
isTrain = 'Test'
path1 = 'CombinedAcc/'+isTrain+'CombinedFile_walking.csv'
path2 = 'CombinedAcc/'+isTrain+'CombinedFile_sitting.csv'
path3 = 'CombinedAcc/'+isTrain+'CombinedFile_standing.csv'
path4 = 'CombinedAcc/'+isTrain+'CombinedFile_laying.csv'
frame = pd.DataFrame()
list_ = []

df = pd.read_csv(path1,index_col=None, header=0)
list_.append(df)
df = pd.read_csv(path2,index_col=None, header=0)
list_.append(df)
df = pd.read_csv(path3,index_col=None, header=0)
list_.append(df)
df = pd.read_csv(path4,index_col=None, header=0)
list_.append(df)

frame = pd.concat(list_)

In [10]:
frame.head
frame.to_csv('CombinedAcc/CombinedFile_'+isTrain+'.csv', sep=',')

In [8]:
frame.head(10)
#df.head(10)

,Unnamed: 0,0,1,2,3,4
0,0,1518920358183,-0.454792,-0.473941,7.137835,walking
1,1,1518920358187,-0.397344,-0.339897,7.080388,walking
2,2,1518920358191,-0.320748,-0.320748,7.080388,walking
3,3,1518920358195,-0.359046,-0.205853,7.137835,walking
4,4,1518920358199,-0.397344,-0.110107,7.176133,walking
5,5,1518920358203,-0.435643,-0.129257,7.156984,walking
6,6,1518920358207,-0.454792,-0.148406,7.156984,walking
7,7,1518920358211,-0.435643,-0.129257,7.118686,walking
8,8,1518920358215,-0.512239,-0.205853,7.061238,walking
9,9,1518920358218,-0.627134,-0.110107,7.099536,walking


In [9]:
fp=open('CombinedAcc/CombinedFile_Train.csv',"r")
wfp=open("CombinedAcc/TrainFinalFile.csv","w")

i = 0
for line in fp:
    i +=1
    if i==1:
        continue;
    #print(line)
    l=line.split(",")
    s=l[6].strip('\n')
    if s=='sitting' or s=='standing' or s=='walking' or s=='laying_down':
        temp_string = l[3]+','+l[4]+','+l[5]+','+l[6] 
        wfp.write(temp_string)


fp.close()
wfp.close()

fp=open('CombinedAcc/CombinedFile_Test.csv',"r")
wfp=open("CombinedAcc/TestFinalFile.csv","w")

i = 0
for line in fp:
    i +=1
    if i==1:
        continue;
    #print(line)
    l=line.split(",")
    s=l[6].strip('\n')
    if s=='sitting' or s=='standing' or s=='walking' or s=='laying_down':
        temp_string = l[3]+','+l[4]+','+l[5]+','+l[6] 
        wfp.write(temp_string)


fp.close()
wfp.close()

In [20]:
# LSTM 
def create_data(filename):
    from scipy import stats
    from sklearn.cross_validation import train_test_split
    columns = [ 'x-axis', 'y-axis', 'z-axis','activity']
    df = pd.read_csv(filename, header = None, names = columns)

    N_TIME_STEPS = 250
    N_FEATURES = 3
    step = 1000
    segments = []
    labels = []
    for i in range(0, len(df) - N_TIME_STEPS, step):
        xs = df['x-axis'].values[i: i + N_TIME_STEPS]
        ys = df['y-axis'].values[i: i + N_TIME_STEPS]
        zs = df['z-axis'].values[i: i + N_TIME_STEPS]
        label = stats.mode(df['activity'][i: i + N_TIME_STEPS])[0][0]
        segments.append([xs, ys, zs])
        labels.append(label)



    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, N_TIME_STEPS, N_FEATURES)
    labels = np.asarray(pd.get_dummies(labels), dtype = np.float32)
    return(reshaped_segments,labels)

X_train, y_train = create_data('CombinedAcc/TrainFinalFile.csv')
X_test, y_test = create_data('CombinedAcc/TestFinalFile.csv')

/home/kamal/miniconda3/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [35]:
N_TIME_STEPS = 250
N_FEATURES = 3
step = 250
input_layer = Input(shape=(N_TIME_STEPS,N_FEATURES,), dtype='float32', name='input_layer')
#cnn_layer = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu',name = 'cnn_layer') (input_layer)
#pool_layer = MaxPooling1D(pool_size = 2,name = 'pool_layer')(cnn_layer)

#cnn_layer = Conv1D(filters=32, kernel_size=3, padding='same', activation='relu',name = 'cnn_layer_1') (pool_layer)
#pool_layer = MaxPooling1D(pool_size = 2,name = 'pool_layer_1')(cnn_layer)

#cnn_layer = Conv1D(filters=8, kernel_size=3, padding='same', activation='relu',name = 'cnn_layer_2') (pool_layer)
#pool_layer = MaxPooling1D(pool_size = 2,name = 'pool_layer_2')(cnn_layer)

#dense_layer = Flatten()(pool_layer)
lstm_layer = GRU(32,dropout=0.2, recurrent_dropout=0.2, name='lstm_layer', return_sequences=True)(input_layer)
lstm_layer = GRU(32,dropout=0.2, recurrent_dropout=0.2, name='lstm_layer_1')(lstm_layer)
dense_layer = Dense(64,activation = 'sigmoid',name = 'dense_1')(lstm_layer)
dense_layer = Dense(4,activation = 'softmax',name='dense_2')(dense_layer)

model = Model(inputs = input_layer,outputs = dense_layer)
print(model.summary())
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train,validation_split = 0.1, epochs=10,batch_size=128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 250, 3)            0         
_________________________________________________________________
lstm_layer (GRU)             (None, 250, 32)           3456      
_________________________________________________________________
lstm_layer_1 (GRU)           (None, 32)                6240      
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 260       
Total params: 12,068
Trainable params: 12,068
Non-trainable params: 0
_________________________________________________________________
None
Train on 12594 samples, validate on 1400 samples
Epoch 1/10
12594/12594 [==============================] - 46s - loss: 1.0773 - acc: 

In [36]:
model.evaluate(X_test,y_test)

302/302 [==============================] - 0s     


[1.1594543591240385, 0.43046357615894038]

In [20]:
y_train[1]

array([ 0.,  0.,  0.,  1.], dtype=float32)